In [ ]:
################ Simple example of word embeedings  ##########################
# It takes 4 words and gets thes embeddings (w1, w2) as it 2d representation #
##############################################################################
import torch
import math
import numpy as np
from matplotlib import pyplot as plt
import torch.nn.functional as F
import re

In [ ]:
torch.manual_seed(298630698)

In [ ]:
words =["Troll","is", "great","Gymkata"]

In [ ]:

L = ["Troll is great\n", "Gymkata is great\n", "Sam is fat and heavy\n", "Sam is great\n","Dali is small\n","Gymkata is fat\n",
     "Dali is great\n", "Dali is a bit small\n","Sam and Dali are heavy\n"]
dictionary=set()
# writing to file
file1 = open('myfile.txt', 'w')
file1.writelines(L)
file1.close()
 
# Using readlines()
file1 = open('myfile.txt', 'r')
file1 = open('texto3.txt', 'r')
Lines = file1.readlines()
 
count = 0
# Strips the newline character
for line in Lines:
    count += 1
    print("Line {}: {}".format(count, line.strip()))
    # using split() method with a regular expression
    # to extract words from string
    res = re.split(r'\W+', line.strip())
    words = []
    for word in res:
        if word:
            words.append(word)
    for word in words:
        #print(word)
        dictionary.add(word)
print(dictionary)
classes= len(dictionary)


In [ ]:
chars = (dictionary)
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [ ]:
xs=[]
ys=[]

In [ ]:
g=[]
i=0
for line in Lines:
    print("frase ",end="")
    print(i,": ",line)
    
    for w in line.split():
     #   print("2 ",w)
        g.append(w)
    for j,l in zip(g,g[1:]):
        print(j,l)
        ix1 = stoi[j]
        ix2 = stoi[l]
        #print(ix1, ix2)
        xs.append(ix1)
        ys.append(ix2)
    i=i+1
    g=[]

In [ ]:
#xs, ys = [], []
#for line in Lines:
#    for j,l in zip(line,line[1:]):
#        print(j,l)
#        ix1 = stoi[j]
#        ix2 = stoi[l]
#        print(ix1, ix2)
#        xs.append(ix1)
#        ys.append(ix2)
    
xs = torch.tensor(xs)
ys = torch.tensor(ys) 

In [ ]:
xs=xs-1

In [ ]:
ys=ys-1

In [ ]:
xenc = F.one_hot(xs, num_classes=classes).float()

In [ ]:
yenc = F.one_hot(ys, num_classes=classes).float()
yenc

In [ ]:
x =  xenc #torch.tensor(((1,0,0,0,0),(0,1,0,0,0),(0,0,1,0,0),(0,0,0,1,0),(0,0,0,0,1)),requires_grad=False,dtype=torch.float32)
w1 = torch.randn((classes,2),requires_grad=True,dtype=torch.float32)
w2 = torch.randn((2,classes),requires_grad=True,dtype=torch.float32)
y =  yenc #torch.tensor(((0,1,0,0,0),(0,0,1,0,0),(0,0,0,1,0),(0,1,0,0,0),(0,1,0,0,0)) ,requires_grad=False,dtype=torch.float32)

In [ ]:
#x = torch.tensor(((1,0,0,0),(0,1,0,0),(0,0,1,0),(0,0,0,1)),requires_grad=False,dtype=torch.float32)
#w1 = torch.randn((4,2),requires_grad=True,dtype=torch.float32)
#w2 = torch.randn((2,4),requires_grad=True,dtype=torch.float32)
#y = torch.tensor(((0,1,0,0),(0,0,1,0),(0,0,0,1),(0,1,0,0)) ,requires_grad=False,dtype=torch.float32)

In [ ]:
x.to("cuda:0")
w1.to("cuda:0")
w2.to("cuda:0")
y.to("cuda:0")

In [ ]:
w1

In [ ]:
w1.retain_grad()
w2.retain_grad()


In [ ]:

for i in range (3000):
    z1= x@ w1
    z2= z1 @ w2
    f= torch.softmax(z2,dim=1)
    loss = torch.nn.CrossEntropyLoss()
    w1.grad=None
    w2.grad =None
    r=0.1
    output = loss(f, y)
    output.backward()
    w1.data+=-r*w1.grad
    w2.data+=-r*w2.grad
print(output)

In [ ]:
z1= x@ w1
z2= z1 @ w2
f= torch.softmax(z2,dim=1)
print(f)

In [ ]:
p=list(x[0])
max_value = max(p)
max_index = p.index(max_value)
itos

In [ ]:
for h in range(15):
    s=list(f[h])
    max_value = max(s)
    max_index = s.index(max_value)
    print(itos[max_index+1])
    


In [ ]:
x


In [ ]:
w1

In [ ]:
d = w1.tolist()
x1,y1 =list(zip(*d))
plt.scatter(x1,y1)
for i, txt in enumerate(dictionary):
    plt.annotate(txt, (x1[i], y1[i]))
plt.show()

In [ ]:
d

In [ ]:
fig = plt.figure()
plt.plot(w1.detach().numpy(), color='blue')
plt.legend(['Train Loss', 'Test Loss'], loc='upper right')
plt.xlabel('number of training examples seen')
fig

In [ ]:
import torch
import math
import numpy as np
from matplotlib import pyplot as plt
import torch.nn.functional as F
import re

In [ ]:
words = open('names.txt', 'r').read().splitlines()

In [ ]:
print(words)

In [ ]:

chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.']=0
itos = {i:s for s,i in stoi.items()}
print(itos)

In [ ]:
block_size=3
context=[0]*block_size

In [ ]:
context

In [ ]:
X,Y = [],[]
for w in words:
    #print("word: ",w)
    for ch in w+'.':
        #print("char: ",ch)
        ix= stoi[ch]
        X.append(context)
        Y.append(ix)
        #print(''.join(itos[i] for i in context) ,"---->", itos[ix])
        context = context[1:] +[ix]

X=torch.tensor(X)
Y=torch.tensor(Y)

                                                            
    


In [ ]:
X


In [ ]:
X.shape

In [ ]:
g= torch.Generator().manual_seed(2147483647)
C= torch.rand((27,2),generator=g)
W1= torch.rand(6,100,generator=g)
b1 = torch.rand(100,generator=g)
W2 = torch.randn((100,27),generator=g)
b2 = torch.randn(27,generator=g)
parameters =[C,W1,b1,W2,b2]

In [ ]:
X.shape

In [ ]:

C.to("cuda:0")
W1.to("cuda:0")
b1.to("cuda:0")
W2.to("cuda:0")
b2.to("cuda:0")


In [ ]:
for p in parameters:
    p.requires_grad= True

In [ ]:
sum(p.nelement() for p in parameters)

In [ ]:
for _ in range(1000):
    emb= C[X]
    h=torch.tanh(emb.view(-1,6) @ W1 + b1)
    logits = h @ W2+ b2
    loss= F.cross_entropy(logits,Y)
    print(loss.item())
    for p in parameters:
            p.grad= None
    loss.backward()
    for p in parameters:
        p.data = p.data -0.1* p.grad

print(loss.item())

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(C[:,0].data,C[:,1].data,s=200)
for i in range(C.shape[0]):
    plt.text(C[i,0].item(),C[i,1].item(),itos[i],color='white',ha='center',va='center')
plt.grid('minor')

In [ ]:
for i in range(20):
    out=[]
    context = [0]*block_size
    while True:
        emb=C[torch.tensor([context])]
        h= torch.tanh(emb.view(1,-1) @ W1+b1)
        logits = h@W2 +b2
        probs = F.softmax(logits,dim=1)
        ix= torch.multinomial(probs, num_samples=1, generator=g).item()
        context = context[1:]+[ix]
        out.append(ix)
        if ix==0:
            break
    print(''.join(itos[i] for i in out))

In [ ]:
################### GPT #############

In [ ]:
import torch

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [ ]:
text = open("input.txt").read()
print(text)

In [ ]:
chars = list(sorted(set(text)))
chars

In [ ]:
for i in enumerate(chars):
    print(i)

In [ ]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda word:    [stoi[ch] for ch in word]
decode = lambda entero:  ''.join([itos[i] for i in entero])

In [ ]:
print(encode("perro"))

In [ ]:
print(decode(encode("perro")))

In [ ]:
data= torch.tensor(encode(text),dtype=torch.long)

In [ ]:
print(data[:100])

In [ ]:
n= int(0.9*len(data))
train_data= data[:n]
val_data= data[n:]

In [ ]:
block_size=8
train_data[:block_size+1]

In [ ]:
x=train_data[:block_size]
y=train_data[1:block_size+1]
for t in range(block_size):
    context= x[:t+1]
    target = y[t]
    print(f"when the input is {context} the target is {target}")

In [ ]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data= train_data if split=="train" else val_data
    ix= torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb,yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)

print('targets:')
print(yb.shape)
print(yb)

                
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f"when the input is {context} the target is {target}")
        

In [ ]:
################ Embedding ############
import torch 
from torch.nn import Embedding

In [ ]:
n_embeddings,dim = 10,4

In [ ]:
emb1 = Embedding(n_embeddings, dim)

In [ ]:
emb1

In [ ]:
emb1.weight

In [ ]:
imp = torch.LongTensor([[1,3],[5,5]])

In [ ]:
emb1(imp)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward0>)


In [3]:
embeds.weight

Parameter containing:
tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519],
        [-0.1661, -1.5228,  0.3817, -1.0276, -0.5631]], requires_grad=True)

In [4]:
class hola():
    def __init__(self,v):
        self.vocal=v
    
    def forward(self,idx):
        return "aa"
        
    def forward1(self,idx):
        return "holaa"
    
    def generate(self,idx):
        f=self(idx)
        return "3"+f
    

d=hola("a")

In [5]:
d.generate(2)

TypeError: 'hola' object is not callable

In [7]:
class myLayer(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer1 = nn.Linear(10,1)
  def forward(self, input_tensor):
    return self.layer1(input_tensor)
model = myLayer()
input_tensor = torch.rand((2,10))
#//treat as callable, which is same as model.forward(tensor)
model(input_tensor)

tensor([[ 0.1101],
        [-0.1957]], grad_fn=<AddmmBackward0>)

In [ ]:
############# Conditional probaility########

In [3]:
import re

In [4]:
def CountFrequency(my_list):

	# Creating an empty dictionary
	freq = {}
	for item in my_list:
		if (item in freq):
			freq[item] += 1
		else:
			freq[item] = 1

	for key, value in freq.items():
		print("% d : % d" % (key, value))


# Driver function
if __name__ == "__main__":
	my_list = [1, 1, 1, 5, 5, 3, 1, 3, 3, 1, 4, 4, 4, 2, 2, 2, 2]

	CountFrequency(my_list)


 1 :  5
 5 :  2
 3 :  3
 4 :  3
 2 :  4


In [8]:
conjunto =set()
file1 = open('texto3.txt', 'r')
Lines = file1.readlines()
 
count = 0
# Strips the newline character
for line in Lines:
    count += 1
    print("Line {}: {}".format(count, line.strip()))
    res = re.split(r'\W+', line.strip())
    words = []
    for word in res:
        if word:
            words.append(word)
    for word in words:
        #print(word)
        conjunto.add(word)
print(conjunto)
classes= len(conjunto)

Line 1: Roger is young
Line 2: Dali is a dog
Line 3: Dakota is old
Line 4: Roger is funny
Line 5: Marta is small
Line 6: Magce is fat
Line 7: Julia is white
Line 8: Julia is working
Line 9: Roger sleeps late
Line 10: Roger is happy
Line 11: NEC is a company
Line 12: Huawei is a company
Line 13: Nokia is a company
Line 14: Dakota is a dog
Line 15: Marta is a woman
Line 16: Magce is a woman
Line 17: Marta is black
{'dog', 'Dakota', 'NEC', 'funny', 'young', 'happy', 'Nokia', 'a', 'is', 'Marta', 'sleeps', 'fat', 'company', 'white', 'black', 'Magce', 'working', 'late', 'woman', 'old', 'Huawei', 'Julia', 'Dali', 'small', 'Roger'}
